In [47]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
df=pd.read_json("/Users/David/code/Clementinelalande/marbotic/raw_data/export_23-5_minified.json")

In [79]:
df.shape

(1320229, 16)

In [80]:
df.columns

Index(['city', 'client_event_time', 'country', 'device_model', 'event_id',
       'event_type', 'is_attribution_event', 'language', 'os_version',
       'region', 'session_id', 'start_version', 'user_creation_time',
       'user_id', 'user_properties', 'event_properties'],
      dtype='object')

In [81]:
df.event_type.unique()

array(['[Activity] Good Answer', '[Activity] Wrong Answer',
       '[Activity] Stop', '[Activity] Start', '[Action] Click',
       '[Scene] Leave', '[Scene] Enter', '[Game] Pause',
       '[Scaffolding] Scaffolding', '[Game] Start', '[Error] Error',
       '[Game] Stop', '[OB] CommPreview', '[OB] DoYouHaveKit',
       '[OB] HasKitYes', '[OB] Email', '[OB] VerifCode',
       '[Activation] Game', '[OB] HasKitNo', '[OB] DiscoverTrial',
       '[OB] ClickTrial', '[Account] Sign In', '[Profile] Create',
       '[Profile] Update', '[OB] ProductSelection', '[OB] ProductHelp',
       '[OB] Type', '[Account] Sign Up', '[OB] ProfileName',
       '[OB] ProfileBirthdate', '[Activation] New Activation',
       '[Activation] Product', '[Activation] Piece', '[OB] Success',
       '[OB] ProductActivation', '[Scaffolding] Reset', '[OB] TrialStart',
       '[Account] Update Request', '[Profile] Delete', '[Popup] Open',
       '[Popup] Close', '[Account] Update Success', '[Account] Sign Out',
       '[Pr

In [82]:
df.head()

,city,client_event_time,country,device_model,event_id,event_type,is_attribution_event,language,os_version,region,session_id,start_version,user_creation_time,user_id,user_properties,event_properties
0,Dallas,2022-03-01 00:57:25.903,United States,"iPad13,4",1110,[Activity] Good Answer,False,English,15.3,Texas,1646095346396,1.0.7,2021-12-28 01:29:38.094,67531,"{'Newsletter': 'False', 'Products': ['Smart Le...","{'[Activity] Name': 'LinePanties', '[Activity]..."
1,Dallas,2022-03-01 00:57:41.878,United States,"iPad13,4",1111,[Activity] Wrong Answer,False,English,15.3,Texas,1646095346396,1.0.7,2021-12-28 01:29:38.094,67531,"{'Newsletter': 'False', 'Products': ['Smart Le...","{'[Activity] Name': 'LinePanties', '[Activity]..."
2,Toronto,2022-03-01 00:57:28.175,Canada,"iPad12,1",1430,[Activity] Stop,False,English,15.2,Ontario,1646096085235,1.3.1,2021-07-19 20:24:36.326,60882,"{'Newsletter': 'True', 'Products': ['Smart Let...","{'[Activity] Name': '1.2', '[Game] Code': 'Mar..."
3,Toronto,2022-03-01 00:57:28.177,Canada,"iPad12,1",1431,[Activity] Start,False,English,15.2,Ontario,1646096085235,1.3.1,2021-07-19 20:24:36.326,60882,"{'Newsletter': 'True', 'Products': ['Smart Let...","{'[Activity] Name': '1.3', '[Game] Code': 'Mar..."
4,Toronto,2022-03-01 00:57:38.278,Canada,"iPad12,1",1432,[Activity] Stop,False,English,15.2,Ontario,1646096085235,1.3.1,2021-07-19 20:24:36.326,60882,"{'Newsletter': 'True', 'Products': ['Smart Let...","{'[Activity] Name': '1.3', '[Game] Code': 'Mar..."


In [83]:
user_properties_df=df.copy()

In [84]:
user_properties=user_properties_df[["user_id","user_properties"]]

In [85]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=PoQsXFrUUsLui0bniQSz9njix6vkmV&access_type=offline&code_challenge=S3iaAqXEe7U5N46YKdeJCKShEw-o-S2tI7TMNTfuXWM&code_challenge_method=S256


You are now logged in as [david.periers@gmail.com].
Your current project is [marbotic].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [86]:
!gcloud projects list

PROJECT_ID              NAME              PROJECT_NUMBER
marbotic                Marbotic          566474310042
symmetric-curve-346916  My First Project  175516577791
wagon-bootcamp-346916   Wagon bootcamp    697736453840


In [87]:
!gcloud config set project marbotic

Updated property [core/project].


In [88]:
!bq ls

/usr/local/Caskroom/google-cloud-sdk/latest/google-cloud-sdk/platform/bq/bq.py:41: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
     datasetId      
 ------------------ 
  marbotic_dataset  


In [89]:
!cat credentials

cat: credentials: No such file or directory


In [93]:
from google.oauth2 import service_account

key_path = "/Users/David/code/dperiers/GITHUB_NICKNAME/gcp/marbotic-587600c2c13f.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [98]:
def extract():
    # Construct a BigQuery client object.
    client = bigquery.Client(project="marbotic", credentials=credentials)

    query = """
        SELECT *
        FROM `marbotic.marbotic_dataset.event_test`
        LIMIT 20
    """

    query_job = client.query(query)  # Make an API request.
    results=query_job.result().to_dataframe() # Waits for job to complete.

    return pd.DataFrame(results)

In [99]:
extract()

,user_id,start_version,os_version,client_event_time,language,event_id,session_id,region,is_attribution_event,user_creation_time,device_model,event_type,country,city
0,NaN,None,NaN,2022-03-01 00:57:38.278000+00:00,None,NaN,NaN,None,None,NaT,None,None,Canada,Toronto
1,NaN,None,NaN,2022-03-01 00:57:38.278000+00:00,None,NaN,NaN,None,None,NaT,iPad Air 2,None,Canada,Toronto
2,NaN,None,NaN,2022-03-01 00:57:38.278000+00:00,None,NaN,NaN,None,None,NaT,None,None,Canada,Toronto
3,NaN,None,NaN,2022-03-01 00:57:38.278000+00:00,None,NaN,NaN,None,None,NaT,None,None,None,Toronto
4,NaN,None,NaN,NaT,None,NaN,NaN,None,None,NaT,None,None,None,Toronto
5,NaN,None,NaN,2022-03-01 00:57:38.278000+00:00,None,23211.0,NaN,None,None,NaT,iPad Air 2,[Scene] Enter,Canada,Toronto
6,NaN,None,NaN,2022-03-01 00:57:38.278000+00:00,None,23211.0,NaN,None,None,NaT,iPad Air 2,None,Canada,Toronto
7,NaN,None,15.1,2022-03-01 00:57:38.278000+00:00,English,23211.0,NaN,None,False,NaT,iPad Air 2,[Scene] Enter,Canada,Toronto
8,NaN,None,15.1,2022-03-01 00:57:38.278000+00:00,English,23211.0,1.646095e+12,New York,False,NaT,iPad Air 2,[Scene] Enter,Canada,Toronto
9,NaN,None,NaN,2022-03-01 00:57:38.278000+00:00,English,23211.0,NaN,None,False,NaT,iPad Air 2,[Scene] Enter,Canada,Toronto


In [ ]:
test=pd.DataFrame(columns=["a","b"])
test